In [ ]:
# Install pre-requisites
!pip install -q -U trl transformers accelerate sentencepiece git+https://github.com/huggingface/peft.git
!pip install -q -U datasets bitsandbytes einops scipy wandb

This notebook was used to calculate the perplexity of meta-llama/Llama-2-7b-chat-hf and compare it to a fine-tuned meta-llama/Llama-2-7b-chat-hf model
https://www.kaggle.com/code/philculliton/calculating-the-perplexity-of-4-bit-llama-2


In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import torch
from transformers import AutoModelForCausalLM, LlamaTokenizer, BitsAndBytesConfig, AutoTokenizer
import bitsandbytes as bnb

model_name = 'meta-llama/Llama-2-7b-chat-hf'
# model_name = "psmathur/orca_mini_3b"
# model_name = "rmuema/orca_mini_3B_test_guanaco"

tokenizer = LlamaTokenizer.from_pretrained(model_name)

free_in_GB = int(torch.cuda.mem_get_info()[0]/1024**3)
max_memory = f'{int(torch.cuda.mem_get_info()[0]/1024**3)-2}GB'

n_gpus = torch.cuda.device_count()
max_memory = {i: max_memory for i in range(n_gpus)}

model = AutoModelForCausalLM.from_pretrained(
 model_name,
 device_map='auto',
 load_in_4bit=True,
 max_memory=max_memory,
 do_sample=True,
 torch_dtype="auto"
)

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
# )

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     quantization_config=bnb_config,
#     trust_remote_code=True
# )
# model.config.use_cache = False

# tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
from datasets import load_dataset

# dataset_name = 'rmuema/Kaggle-X-ELO-QA'
dataset_name = 'rmuema/kaggleX-elo'
test = load_dataset(dataset_name, split="train")
# dataset = load_dataset(dataset_name)
# test = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
encodings = tokenizer("\n\n".join(test["text"]), return_tensors="pt")

In [15]:
import torch
from tqdm import tqdm

max_length = model.config.max_length
stride = 512
seq_len = encodings.input_ids.size(1)
device = "cuda"

nlls = []
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

        # loss is calculated using CrossEntropyLoss which averages over valid labels
        # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
        # to the left by 1.
        neg_log_likelihood = outputs.loss

    nlls.append(neg_log_likelihood)

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break

ppl = torch.exp(torch.stack(nlls).mean())

100%|██████████| 1573/1573 [11:17<00:00,  2.32it/s]


In [16]:
ppl

tensor(163.4296, device='cuda:0')

In [7]:
# Calculate PPL for Fine Tuned Model
# base_model = "psmathur/orca_mini_3b"
base_model = "meta-llama/Llama-2-7b-chat-hf"
model_name = "rmuema/kaggle-x-elo-finetune-v1.2"

tokenizer = LlamaTokenizer.from_pretrained(base_model)

free_in_GB = int(torch.cuda.mem_get_info()[0]/1024**3)
max_memory = f'{int(torch.cuda.mem_get_info()[0]/1024**3)-2}GB'

n_gpus = torch.cuda.device_count()
max_memory = {i: max_memory for i in range(n_gpus)}

model = AutoModelForCausalLM.from_pretrained(
 base_model,
 device_map='auto',
 load_in_4bit=True,
 max_memory=max_memory,
 do_sample=True,
 torch_dtype="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig.from_pretrained(model_name)
infer_model = get_peft_model(model, lora_config)

In [9]:
max_length = model.config.max_length
stride = 512
seq_len = encodings.input_ids.size(1)
device = "cuda"

nlls = []
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = infer_model(input_ids, labels=target_ids)

        # loss is calculated using CrossEntropyLoss which averages over valid labels
        # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
        # to the left by 1.
        neg_log_likelihood = outputs.loss

    nlls.append(neg_log_likelihood)

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break

ppl = torch.exp(torch.stack(nlls).mean())

100%|██████████| 1573/1573 [11:16<00:00,  2.33it/s]


In [10]:
ppl

tensor(163.4296, device='cuda:0')